In [2]:
import chess.pgn
import sys

f1 = open('white_wins.txt', 'w')
f2 = open('black_wins.txt', 'w')
pgn = open("train.pgn")
n_games = 0

while True:
    game = chess.pgn.read_game(pgn)
    try:
        result = game.headers["Result"]
    except:
        break

    if result == "1-0":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f1.write(str(board))
            f1.write("\n- - - - - - - -\n")
        f1.write("\n= = = = = = = =\n")
    if result == "0-1":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f2.write(str(board))
            f2.write("\n- - - - - - - -\n")
        f2.write("\n= = = = = = = =\n")

f1.close()        
f2.close()
pgn.close()
print("Done. ", n_games, " games parsed")

ModuleNotFoundError: No module named 'chess'

In [3]:
import tensorflow as tf
white = open('white_wins.txt', 'r')
black = open('black_wins.txt', 'r')



In [4]:
import numpy as np

# convert all data to a list form 
i = 0
boards = []
for line in white:
    temp = line.strip().split(" ")
    boards.append(temp)
# tst = np.load("A00-139_first_10000.npz")
# boards = tst['arr_0']
# targets = tst['arr_1']
# print(boards.shape)
# print(boards[0,1,:,:])
# print(targets.shape)
# print(boards)
boards_data = np.asarray(boards)

black_boards = []
for line in black:
    temp = line.strip().split(" ")
    black_boards.append(temp)


In [5]:
# First lets create a 8x8 boards
i = 0
boards_final = []
checklist = ['r','n', 'b', 'q', 'k', 'p', 'R','N', 'B', 'Q', 'K', 'P', '.']

for i in range(len(boards_data)):
    if any(check in boards_data[i] for check in checklist):
        boards_final.append(boards_data[i])
    
# print(len(boards_final))
# print(boards_final)
bbf = []
bb = np.asarray(black_boards)
for i in range(len(bb)):
    if any(check in bb[i] for check in checklist):
        bbf.append(bb[i])

In [6]:
# restructure the white boards to a chess board shape
boards_np = np.asarray(boards_final)
boards_np = boards_np.reshape((int(boards_np.shape[0]/8),8,8))

# restructure the black boards data to a chess board shape

bbf_np = np.asarray(bbf)
bbf_np = bbf_np.reshape((int(bbf_np.shape[0]/8),8,8))

# print(boards_np.shape)

In [9]:
# print(boards_np[0])
# convert to a map of 6x8x8 data type with the extra channel for 
# mapping the 6 types of chess pieces
boards_res = []

for i in range(len(boards_np)):
    test = boards_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    boards_res.append(temp)

    
boards_res = np.asarray(boards_res)
print(boards_res.shape)
# print(boards_res)
# print(temp1)
# print(temp2)
# print(temp3)
# print(temp4)
# print(temp5)
# print(temp6)

(216982, 6, 8, 8)


In [ ]:
black_res = []

for i in range(len(bbf_np)):
    test = bbf_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    black_res.append(temp)

    
black_res = np.asarray(black_res)
print(black_res.shape)
# print(black_res)

In [110]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# NOTE: maybe try training on complete games instead of single board positions
# NOTE: (-1, -1, 6, 8, 8)


# boards_res is of the form (216982,6,8,8) which is the correct shape for white wins
white_input_data = boards_res # board position where white wins
black_input_data = black_res # board position where black wins
white_input_label = np.ones(boards_res.shape[0])
black_input_label = (np.zeros(black_res.shape[0]))
print(white_input_label.shape)
print(black_input_label.shape)
input_label = np.int32(np.concatenate((white_input_label, black_input_label)))
print(input_label.shape)
# print(input_label)
input_data = np.float32(np.concatenate((white_input_data, black_input_data)))
print(input_data.shape)
# print(type(input_data[0][0][0][0]))

inp_data, eval_data, inp_label, eval_label = train_test_split(input_data, input_label, test_size=0.2, random_state=42)

# print(inp_data.shape)
# print(eval_data.shape)
# print(inp_label.shape)
# print(eval_label.shape)

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_fn(features, labels, mode):
    # Convolution layer #1
    input_layer = tf.reshape(features, [-1,6,8,8])
    conv1 = tf.layers.conv2d(inputs=input_layer, 
                             filters=64,
                             kernel_size=[5,5],
                             padding="same",
                             activation=tf.nn.relu)
#     print(conv1.shape)
    # Pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

#     print(pool1.shape)
    # print(type(pool1))

    # Convolution layer #2
    conv2 = tf.layers.conv2d(inputs=dropout1,
                            filters=32,
                            kernel_size=[3,3],
                            padding="same",
                            activation=tf.nn.relu)
#     print(conv2.shape)
    # print(type(conv2))
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)

    print(pool2.shape)
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 1*2*32])
    dense = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

#     print("Dense:", dense.shape)
    # Logit layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
#     print("Logits:", logits.shape)
    # print(logits)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    # restructure input data 
    labels = input_label
    # setting the mode to training right now
    # mode = tf.estimator.ModeKeys.TRAIN

    cnn_classifier = tf.estimator.Estimator(model_fn=cnn_fn, model_dir="cnn_model")

    tensor_to_log = {"probabilites":"softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=1000)
    
    # batch_size = 1000 takes about 280 sec (a little too much for my computer)
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=inp_data,
        y=inp_label,
        batch_size=100,
        num_epochs=None,
        shuffle=True)

    cnn_classifier.train(input_fn=train_input_fn,
                          steps=10000,
                          hooks=[logging_hook])

    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=eval_data,
        y=eval_label,
        num_epochs=1,
        shuffle=False)
    eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()

# cnn_fn(input_data, input_label, mode)


(216982,)
(201837,)
(418819,)
(418819, 6, 8, 8)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000197A8E8ACC0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(100, 1, 2, 32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn_model\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:loss = 0.5052959, step = 22001 (3.066 sec)
INFO:tensorflow:global_step/sec: 56.454
INFO:tensorflow:loss = 0.4387201, step = 22101 (1.768 sec)
INFO:tensorflow:global_step/sec: 52.1877
INFO:tensorflow:loss = 0.5959944, step = 22201 (1.916 sec)
INFO:tensorflow:global_step/sec: 23.2395
INFO:tensorflow:loss = 0.62449247, step = 22301 (4.307 sec)
INFO:tensorflow:global_step/sec: 37.9112
INFO:tensorflow:loss = 0.55080587, step = 22401 (2.634 sec)
INFO:tensorflow:global_step/sec: 56.5753
INFO:tensorflow:loss = 0.5231217, step = 22501 (1.768 sec)
INFO:tensorflow:global_step/sec: 57.5123
INFO:tensorflow:loss = 0.53838813, step = 22601 (1.743 sec)
INFO:tensorflow:global_step/sec: 27.5007
INFO:tensorflow:loss = 0.57802784, step = 22701 (3.636 sec)
INFO:tensorflow:global_step/sec: 31.5254
INFO:tensorflow:loss = 0.53676724, step = 22801 (3.168 sec)
INFO:tensorflow:global_step/sec: 56.6913
INFO:tensorflow:loss = 0.5638755, step = 22901 (1.764 sec)
INFO:tensorflow:global_step/sec: 57.3

INFO:tensorflow:loss = 0.50015235, step = 25001 (1.741 sec)
INFO:tensorflow:global_step/sec: 56.1861
INFO:tensorflow:loss = 0.46286434, step = 25101 (1.772 sec)
INFO:tensorflow:global_step/sec: 19.1111
INFO:tensorflow:loss = 0.5085124, step = 25201 (5.233 sec)
INFO:tensorflow:global_step/sec: 33.4857
INFO:tensorflow:loss = 0.48634478, step = 25301 (2.986 sec)
INFO:tensorflow:global_step/sec: 57.4677
INFO:tensorflow:loss = 0.5347474, step = 25401 (1.744 sec)
INFO:tensorflow:global_step/sec: 55.6966
INFO:tensorflow:loss = 0.593108, step = 25501 (1.792 sec)
INFO:tensorflow:global_step/sec: 28.8343
INFO:tensorflow:loss = 0.5086019, step = 25601 (3.468 sec)
INFO:tensorflow:global_step/sec: 22.5303
INFO:tensorflow:loss = 0.44913146, step = 25701 (4.438 sec)
INFO:tensorflow:global_step/sec: 57.7768
INFO:tensorflow:loss = 0.50184345, step = 25801 (1.731 sec)
INFO:tensorflow:global_step/sec: 56.1852
INFO:tensorflow:loss = 0.5590854, step = 25901 (1.784 sec)
INFO:tensorflow:global_step/sec: 53.0

INFO:tensorflow:loss = 0.53308827, step = 28001 (2.247 sec)
INFO:tensorflow:global_step/sec: 47.5331
INFO:tensorflow:loss = 0.5031166, step = 28101 (2.097 sec)
INFO:tensorflow:global_step/sec: 55.4334
INFO:tensorflow:loss = 0.5783499, step = 28201 (1.804 sec)
INFO:tensorflow:global_step/sec: 55.1888
INFO:tensorflow:loss = 0.5726006, step = 28301 (1.812 sec)
INFO:tensorflow:global_step/sec: 54.5568
INFO:tensorflow:loss = 0.487604, step = 28401 (1.832 sec)
INFO:tensorflow:global_step/sec: 54.7958
INFO:tensorflow:loss = 0.42587402, step = 28501 (1.828 sec)
INFO:tensorflow:global_step/sec: 49.0958
INFO:tensorflow:loss = 0.5000994, step = 28601 (2.035 sec)
INFO:tensorflow:global_step/sec: 55.0067
INFO:tensorflow:loss = 0.41908, step = 28701 (1.818 sec)
INFO:tensorflow:global_step/sec: 54.1435
INFO:tensorflow:loss = 0.53261286, step = 28801 (1.846 sec)
INFO:tensorflow:global_step/sec: 53.6212
INFO:tensorflow:loss = 0.43183503, step = 28901 (1.866 sec)
INFO:tensorflow:global_step/sec: 52.634


SystemExit: 

C:\Users\Shashank\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
